In [1]:
# In Terminal, "pip install ibis-framework[duckdb] pyjanitor"
import pandas as pd
import ibis
from ibis import selectors as s
from ibis import _
ibis.options.interactive = True

In [2]:
from janitor import clean_names

In [3]:
# Path
from pathlib import Path
path = Path("~/datasets/home-dataset/jupyterlab/ZX05")
db_path = path / "db"
meta_path = path / "meta"
output_path = path / "output"

In [4]:
# Read DB file
# pl_2020 = pd.read_csv(path + "db/PL_costs/PL_2020.csv", dtype="str").clean_names()
# pl_2021 = pd.read_csv(path + "db/PL_costs/PL_2021.csv", dtype="str").clean_names()
# pl_2022 = pd.read_csv(path + "db/PL_costs/PL_2022.csv", dtype="str").clean_names()
pl_2024 = pd.read_csv(db_path / "PL_2024.csv")
df_pl = pd.concat([pl_2024]) # pl_2020, pl_2021, pl_2022, 
df_pl

,fy,period,costctr,gl_accounts,actual,plan,target
0,2024,1,IC-99LD,K66270,27005141,27584485,27584485
1,2024,1,IC-99LD,K66271,184178530,188129602,188129602
2,2024,1,IC-99LD,K66273,113885517,116328686,116328686
3,2024,1,IC-99LD,K66275,4795603,4898472,4898472
4,2024,1,IC-99LD,K66278,188203298,192240828,192240828
...,...,...,...,...,...,...,...
12180,2024,2,6995,S90717,24664,0,0
12181,2024,2,CY-999MK,S87453,668138,744405,744405
12182,2024,2,CY-999MK,S90808,-668138,0,0
12183,2024,2,CY-599MK,S87561,5846237,6153861,6153861


In [5]:
pl = ibis.memtable(df_pl)
pl.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual    ┃ plan      ┃ target    ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ int64     │ int64     │ int64     │
├───────┼────────┼─────────┼─────────────┼───────────┼───────────┼───────────┤
│  2024 │      1 │ IC-99LD │ K66270      │  27005141 │  27584485 │  27584485 │
│  2024 │      1 │ IC-99LD │ K66271      │ 184178530 │ 188129602 │ 188129602 │
│  2024 │      1 │ IC-99LD │ K66273      │ 113885517 │ 116328686 │ 116328686 │
└───────┴────────┴─────────┴─────────────┴───────────┴───────────┴───────────┘

In [6]:
# Process numeric columns
pl = (pl
    .mutate(s.across(["actual", "plan", "target"], _ / -1000))
    .mutate(delta_to_plan = (_.actual - _.plan).round(3))
)    
pl.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual      ┃ plan        ┃ target      ┃ delta_to_plan ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64     │ float64     │ float64     │ float64       │
├───────┼────────┼─────────┼─────────────┼─────────────┼─────────────┼─────────────┼───────────────┤
│  2024 │      1 │ IC-99LD │ K66270      │  -27005.141 │  -27584.485 │  -27584.485 │       579.344 │
│  2024 │      1 │ IC-99LD │ K66271      │ -184178.530 │ -188129.602 │ -188129.602 │      3951.072 │
│  2024 │      1 │ IC-99LD │ K66273      │ -113885.517 │ -116328.686 │ -116328.686 │      2443.169 │
└───────┴────────┴─────────┴─────────────┴─────────────┴─────────────┴─────────────┴───────────────┘

In [7]:
# Add volume difference
pl = pl.mutate(
    volume_difference = (_.plan - _.target).round(3)
)
pl.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual      ┃ plan        ┃ target      ┃ delta_to_plan ┃ volume_difference ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64     │ float64     │ float64     │ float64       │ float64           │
├───────┼────────┼─────────┼─────────────┼─────────────┼─────────────┼─────────────┼───────────────┼───────────────────┤
│  2024 │      1 │ IC-99LD │ K66270      │  -27005.141 │  -27584.485 │  -27584.485 │       579.344 │               0.0 │
│  2024 │      1 │ IC-99LD │ K66271      │ -184178.530 │ -188129.602 │ -188129.602 │      3951.072 │               0.0 │
│  2024 │      1 │ IC-99LD │ K66273      │ -113885.517 │ -116328.686 │ -116328.686 │      2443.169 │               0.0 │
└───────┴────────┴─────────┴─────────────┴─────────────┴─────────────┴─────────────┴───────────────┴───────────────────┘

### Read master data

In [8]:
# Read master data
df_acc = pd.read_csv(meta_path / "0000_TABLE_MASTER_Acc level.csv", dtype=str).clean_names()
df_cc_general = pd.read_csv(meta_path / "0000_TABLE_MASTER_Cost center_general.csv", dtype=str).clean_names()
df_cc_hierarchy = pd.read_csv(meta_path / "0000_TABLE_MASTER_Cost center_hierarchy.csv", dtype=str).clean_names()
df_coom = pd.read_csv(meta_path / "0004_TABLE_MASTER_COOM_2023.csv", dtype=str, usecols=[0, 1, 2]).clean_names()
df_poc = pd.read_csv(meta_path / "POC.csv", dtype=str).clean_names()

In [9]:
acc = ibis.memtable(df_acc)
acc.head(3)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ account_no_ ┃ account_name         ┃ account_description         ┃ acc_lv1                                ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string      │ string               │ string                      │ string                                 │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │
├─────────────┼──────────────────────┼─────────────────────────────┼────────────────────────────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┤
│ K275        │ Vacation payments    │ K275 Vacation payments      │ 158 Social benefit rate wages variable │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │
│ K28401      │ Company pension - w  │ K28401 Company pension - w  │ 158 Social benefit rate wages variable │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │
│ K28811      │ Soc.sec. pens.fund w │ K28811 Soc.sec. pens.fund w │ 158 Social benefit rate wages variable │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │
└─────────────┴──────────────────────┴─────────────────────────────┴────────────────────────────────────────┴─────────────────────────┴───────────────────────┴───────────────────────────┴──────────────┴─────────────────────────────┴─────────────────────────────┴───────────────────────┴───────────────┘

In [10]:
cc_general = ibis.memtable(df_cc_general)
cc_general.head(3)

┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ validity ┃ pctr      ┃ fix_var ┃ cctr_description         ┃ responsible   ┃ manning_type ┃ line_type ┃ department ┃ function ┃ ordera ┃ orderb ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string │ string   │ string    │ string  │ string                   │ string        │ string       │ string    │ string     │ string   │ string │ string │
├────────┼──────────┼───────────┼─────────┼──────────────────────────┼───────────────┼──────────────┼───────────┼────────────┼──────────┼────────┼────────┤
│ 36098  │ Valid    │ 50803-010 │ Var     │ E CT DTC SMD Technician  │ Yang, JunHyuk │ HEF          │ SMD       │ Production │ SMD      │ J      │ JA     │
│ 38899  │ Valid    │ 50803-026 │ Fix     │ P HT HYD Line share cost │ Kim, Hagjae   │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │
│ 34855  │ Valid    │ 50803-034 │ Fix     │ P SC MES ICH Dummy       │ Seo, Jiyun    │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │
└────────┴──────────┴───────────┴─────────┴──────────────────────────┴───────────────┴──────────────┴───────────┴────────────┴──────────┴────────┴────────┘

In [11]:
cc_hierarchy = ibis.memtable(df_cc_hierarchy)
cc_hierarchy.head(3)

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3      ┃ lv4        ┃ lv5    ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string │ string │ string  │ string     │ string   │ string     │ string │
├────────┼────────┼────────┼─────────┼────────────┼──────────┼────────────┼────────┤
│ 4311   │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-MGK │ ICLM-MGK-A │ NULL   │
│ 4321   │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-MGK │ ICLM-MGK-A │ NULL   │
│ 4830   │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-MGK │ ICLM-MGK-A │ NULL   │
└────────┴────────┴────────┴─────────┴────────────┴──────────┴────────────┴────────┘

In [12]:
cc = cc_general.join(cc_hierarchy, "cctr", how="left")
cc.head(3)

┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ validity ┃ pctr      ┃ fix_var ┃ cctr_description         ┃ responsible   ┃ manning_type ┃ line_type ┃ department ┃ function ┃ ordera ┃ orderb ┃ cctr_right ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3      ┃ lv4        ┃ lv5    ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string   │ string    │ string  │ string                   │ string        │ string       │ string    │ string     │ string   │ string │ string │ string     │ string │ string │ string  │ string     │ string   │ string     │ string │
├────────┼──────────┼───────────┼─────────┼──────────────────────────┼───────────────┼──────────────┼───────────┼────────────┼──────────┼────────┼────────┼────────────┼────────┼────────┼─────────┼────────────┼──────────┼────────────┼────────┤
│ 36098  │ Valid    │ 50803-010 │ Var     │ E CT DTC SMD Technician  │ Yang, JunHyuk │ HEF          │ SMD       │ Production │ SMD      │ J      │ JA     │ 36098      │ 45292  │ 45657  │ IC-OICP │ ICLD-O7821 │ ICLD-FGK │ ICLD-FGK-S │ NULL   │
│ 38899  │ Valid    │ 50803-026 │ Fix     │ P HT HYD Line share cost │ Kim, Hagjae   │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │ 38899      │ 45292  │ 45657  │ IC-OICP │ ICEY-O7411 │ ICEY-FGK │ ICEY-FGK-S │ NULL   │
│ 34855  │ Valid    │ 50803-034 │ Fix     │ P SC MES ICH Dummy       │ Seo, Jiyun    │ NULL         │ NULL      │ Production │ FF1      │ J      │ JB     │ 34855      │ 45292  │ 45657  │ IC-OICP │ ICMK-O7521 │ ICMK-FGK │ ICMK-FGK-S │ NULL   │
└────────┴──────────┴───────────┴─────────┴──────────────────────────┴───────────────┴──────────────┴───────────┴────────────┴──────────┴────────┴────────┴────────────┴────────┴────────┴─────────┴────────────┴──────────┴────────────┴────────┘

In [13]:
coom = ibis.memtable(df_coom)
coom.head(3)

┏━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┓
┃ cctr   ┃ account_no_ ┃ coom   ┃
┡━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string      │ string │
├────────┼─────────────┼────────┤
│ 39211  │ K685        │ Var    │
│ 4280   │ K685        │ Var    │
│ 1321   │ K05501      │ Var    │
└────────┴─────────────┴────────┘

In [14]:
poc = ibis.memtable(df_poc)
poc.head(3)

┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name     ┃ outlet_name     ┃ profit_center ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string │ string │ string │ string   │ string         │ string          │ string        │
├────────┼────────┼────────┼────────┼──────────┼────────────────┼─────────────────┼───────────────┤
│ 698    │ 9      │ 9999   │ VT     │ VT       │ none           │ Non-Operational │ 50899-001     │
│ 698    │ 483    │ 7110   │ VT     │ VT       │ ICH Icheon NPF │ CGF             │ 50803-320     │
│ 698    │ 9      │ 7111   │ VT     │ VT       │ none           │ COT             │ 50803-322     │
└────────┴────────┴────────┴────────┴──────────┴────────────────┴─────────────────┴───────────────┘

### Data wrangling

In [15]:
# Join tables to get master data
pl = (pl
    .join(acc, pl.gl_accounts == acc.account_no_, how="left")
    .join(cc, pl.costctr == cc.cctr, how="left")
    .join(poc, cc.pctr == poc.profit_center, how="left")         
    .drop("cctr", "account_no_", "cctr_right", "pctr")
)
pl = (pl
    .join(coom, (pl.costctr == coom.cctr) & (pl.gl_accounts == coom.account_no_), how="left")
    .drop("cctr", "account_no_")
)
pl.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ account_description       ┃ acc_lv1         ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ fix_var ┃ cctr_description        ┃ responsible  ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ float64           │ string               │ string                    │ string          │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string  │ string                  │ string       │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼───────────────────┼──────────────────────┼───────────────────────────┼─────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼─────────┼─────────────────────────┼──────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┤
│  2024 │      2 │ 1321    │ K202        │ -16807.0 │ -16555.974 │ -15448.181 │      -251.026 │         -1107.793 │ Auxiliary wage var.  │ K202 Auxiliary wage var.  │ 175 Total wages │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ Var     │ E CT ENC W/H mgt.-Ship. │ Song, Chomin │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-VK │ NULL   │ NULL   │ 6

In [16]:
# Process COOM data for fix and variable costs
pl = pl.mutate(
    coom = ibis.case()
    .when((pl.fix_var == "Var") & (pl.gl_accounts == "K399"), "Var")
    .when(pl.coom == ibis.NA, "Fix")
    .when(True, pl.coom).end()
)
pl.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ account_description       ┃ acc_lv1         ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ fix_var ┃ cctr_description        ┃ responsible  ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ float64           │ string               │ string                    │ string          │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string  │ string                  │ string       │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼───────────────────┼──────────────────────┼───────────────────────────┼─────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼─────────┼─────────────────────────┼──────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┤
│  2024 │      2 │ 1321    │ K202        │ -16807.0 │ -16555.974 │ -15448.181 │      -251.026 │         -1107.793 │ Auxiliary wage var.  │ K202 Auxiliary wage var.  │ 175 Total wages │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ Var     │ E CT ENC W/H mgt.-Ship. │ Song, Chomin │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292  │ 45657  │ IC-OICP │ ICLM-O7811 │ ICLM-VK │ NULL   │ NULL   │ 6

In [17]:
# Extract function from cost center hierarchy level 3
pl = pl.mutate(
    function_2 = _.lv3.split("-")[1]
)
pl.head(3)


┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ account_description       ┃ acc_lv1         ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ fix_var ┃ cctr_description        ┃ responsible  ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃ function_2 ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ float64           │ string               │ string                    │ string          │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string  │ string                  │ string       │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │ string     │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼───────────────────┼──────────────────────┼───────────────────────────┼─────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼─────────┼─────────────────────────┼──────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┼────────────┤
│  2024 │      2 │ 1321    │ K202        │ -16807.0 │ -16555.974 │ -15448.181 │      -251.026 │         -1107.793 │ Auxiliary wage var.  │ K202 Auxiliary wage var.  │ 175 Total wages │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ Var     │ E CT ENC W/H mgt.-Ship. │ Song, Chomin │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292

### Overtime costs

In [18]:
# Filter variable costs
pl_var = pl.filter(_.coom == "Var")
pl_var.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual   ┃ plan       ┃ target     ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ account_description       ┃ acc_lv1         ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ fix_var ┃ cctr_description        ┃ responsible  ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃ function_2 ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64  │ float64    │ float64    │ float64       │ float64           │ string               │ string                    │ string          │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string  │ string                  │ string       │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │ string     │
├───────┼────────┼─────────┼─────────────┼──────────┼────────────┼────────────┼───────────────┼───────────────────┼──────────────────────┼───────────────────────────┼─────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼─────────┼─────────────────────────┼──────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┼────────────┤
│  2024 │      2 │ 1321    │ K202        │ -16807.0 │ -16555.974 │ -15448.181 │      -251.026 │         -1107.793 │ Auxiliary wage var.  │ K202 Auxiliary wage var.  │ 175 Total wages │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ Var     │ E CT ENC W/H mgt.-Ship. │ Song, Chomin │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292

In [19]:
# Filter overtime_accounts
overtime_accounts = [
    # Variable CC
    "K250", "K256",
    # Fix CC
    "K2501", "K2561",
    # Central function
    "K301", "K302",
]
pl_var = pl_var.filter(_.gl_accounts.isin(overtime_accounts))
pl_var.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual  ┃ plan     ┃ target   ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ account_description       ┃ acc_lv1         ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ fix_var ┃ cctr_description        ┃ responsible  ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃ function_2 ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64 │ float64  │ float64  │ float64       │ float64           │ string               │ string                    │ string          │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string  │ string                  │ string       │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │ string     │
├───────┼────────┼─────────┼─────────────┼─────────┼──────────┼──────────┼───────────────┼───────────────────┼──────────────────────┼───────────────────────────┼─────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼─────────┼─────────────────────────┼──────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┼────────────┤
│  2024 │      2 │ 1322    │ K250        │  -887.2 │ -850.496 │ -792.532 │       -36.704 │           -57.964 │ Shift prem.wages var │ K250 Shift prem.wages var │ 175 Total wages │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ Var     │ E CT DTC W/H mgt.-Ship. │ Song, Chomin │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292  │ 45657  │ IC-OICP │ ICLD-O7

In [20]:
# Add LDC / MDC
pl_var = pl_var.mutate(
    ldc_mdc = ibis.case()
        .when(pl_var.costctr[:1] == "8", "Start up costs")  # CC that starts with "8"
        .when((pl_var.function_2 == "FGK") & (pl_var.acc_lv2 == "299 Total Labor Costs"), "LDC")
        .when((pl_var.function_2 == "FGK") & (pl_var.acc_lv2 == "465 Cost of materials"), "MDC")
        .when(True, "NA").end()
)

In [21]:
# Add account information
pl_var = pl_var.mutate(
    ce_text = ibis.case()
    # LDC
    .when((pl_var.gl_accounts == "K250") | (pl_var.gl_accounts == "K256"), "120 Premium wages")
    .when(pl_var.acc_lv1 == "158 Social benefit rate wages variable", "158 SLB wages")
    .when(pl_var.acc_lv2 == "299 Total Labor Costs", "115 Direct labor")
    # MDC
    .when(True, pl_var.acc_lv1).end()
)
pl_var.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual  ┃ plan     ┃ target   ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ account_description       ┃ acc_lv1         ┃ acc_lv1_by_consolidated ┃ acc_lv2               ┃ acc_lv3                   ┃ acc_lv4      ┃ acc_lv5                     ┃ acc_lv6                     ┃ general_lv1           ┃ pmme_category ┃ validity ┃ fix_var ┃ cctr_description        ┃ responsible  ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃ function_2 ┃ ldc_mdc ┃ ce_text           ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64 │ float64  │ float64  │ float64       │ float64           │ string               │ string                    │ string          │ string                  │ string                │ string                    │ string       │ string                      │ string                      │ string                │ string        │ string   │ string  │ string                  │ string       │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │ string     │ string  │ string            │
├───────┼────────┼─────────┼─────────────┼─────────┼──────────┼──────────┼───────────────┼───────────────────┼──────────────────────┼───────────────────────────┼─────────────────┼─────────────────────────┼───────────────────────┼───────────────────────────┼──────────────┼─────────────────────────────┼─────────────────────────────┼───────────────────────┼───────────────┼──────────┼─────────┼─────────────────────────┼──────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┼────────────┼─────────┼───────────────────┤
│  2024 │      2 │ 1322    │ K250        │  -887.2 │ -850.496 │ -792.532 │       -36.704 │           -57.964 │ Shift prem.wages var │ K250 Shift prem.wages var │ 175 Total wages │ 299 Total Labor Costs   │ 299 Total Labor Costs │ 500 Dir.cost centre costs │ 550 CC costs │ 600 CC cost after All./Ass. │ 610 over-/under- absorption │ 299 Total Labor Costs │ Compensation  │ Valid    │ Var     │ E

In [22]:
# Remove unnecessary columns
pl_var = pl_var.drop(
                    "validity", "responsible", "account_description", "acc_lv1_by_consolidated",
                    "acc_lv3", "acc_lv4", "acc_lv5", "acc_lv6"
)
pl_var.head(3)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ fy    ┃ period ┃ costctr ┃ gl_accounts ┃ actual  ┃ plan     ┃ target   ┃ delta_to_plan ┃ volume_difference ┃ account_name         ┃ acc_lv1         ┃ acc_lv2               ┃ general_lv1           ┃ pmme_category ┃ fix_var ┃ cctr_description        ┃ manning_type ┃ line_type ┃ department ┃ function       ┃ ordera ┃ orderb ┃ from   ┃ to     ┃ lv1     ┃ lv2        ┃ lv3     ┃ lv4    ┃ lv5    ┃ cu     ┃ plant  ┃ outlet ┃ bu     ┃ division ┃ plant_name ┃ outlet_name ┃ profit_center ┃ coom   ┃ function_2 ┃ ldc_mdc ┃ ce_text           ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string      │ float64 │ float64  │ float64  │ float64       │ float64           │ string               │ string          │ string                │ string                │ string        │ string  │ string                  │ string       │ string    │ string     │ string         │ string │ string │ string │ string │ string  │ string     │ string  │ string │ string │ string │ string │ string │ string │ string   │ string     │ string      │ string        │ string │ string     │ string  │ string            │
├───────┼────────┼─────────┼─────────────┼─────────┼──────────┼──────────┼───────────────┼───────────────────┼──────────────────────┼─────────────────┼───────────────────────┼───────────────────────┼───────────────┼─────────┼─────────────────────────┼──────────────┼───────────┼────────────┼────────────────┼────────┼────────┼────────┼────────┼─────────┼────────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────────┼─────────────┼───────────────┼────────┼────────────┼─────────┼───────────────────┤
│  2024 │      2 │ 1322    │ K250        │  -887.2 │ -850.496 │ -792.532 │       -36.704 │           -57.964 │ Shift prem.wages var │ 175 Total wages │ 299 Total Labor Costs │ 299 Total Labor Costs │ Compensation  │ Var     │ E CT DTC W/H mgt.-Ship. │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292  │ 45657  │ IC-OICP │ ICLD-O7821 │ ICLD-VK │ NULL   │ NULL   │ 698    │ 242    │ 7821   │ CT     │ E        │ ICH Icheon │ PL DTC      │ 50803-010     │ Var    │ VK         │ NA      │ 120 Premium wages │
│  2024 │      2 │ 1323    │ K250        │  -459.3 │   -0.000 │   -0.000 │      -459.300 │             0.000 │ Shift prem.wages var │ 175 Total wages │ 299 Total Labor Costs │ 299 Total Labor Costs │ Compensation  │ Var     │ P AC EAC W/H mgt.-Ship. │ HEF          │ NULL      │ SCM        │ W/H mgt.-Ship. │ L      │ LE     │ 45292  │ 45657  │ IC-OICP │ ICTC-O7311 │ ICTC-VK │ NULL   │ NULL   │ 698    │ 242    │ 7311   │ AC     │ P        │ ICH Icheon │ PL EAC      │ 50803-009     │ Var    │ VK         │ NA      │ 120 Premium wages │
│  2024 │      2 │ 1323    │ K256        │  -285.7 │   -0.000 │   -0.000 │      -285.700 │             0.000 │ Overt.prem.wages var │ 175 Total wages │ 299 Total Labor Costs │ 299 Total Labor Costs │ Compensation  │ Var     │ P AC EAC

### Output data

In [23]:
# Output data
pl_var.to_pandas().to_csv(output_path / "PL var OT costs 2020-2024 YTD.csv", index=False)